In [1]:
import os.path
import pandas as pd

# 读取上传的CSV文件
file_path_compare = './compare_output/command_compare_data/1_162.xlsx'
data_compare = pd.read_excel(file_path_compare)
# 显示数据的前几行以了解其结构
data_compare

,number,id,original_company,command_compare_url,command_compare_name
0,1,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_pk/91628e9c2d32f2d6...,技术总监 技术部总经理
1,1,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_pk/149d3e5f7e56b1ab...,技术总监 技术部总经理
2,1,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_pk/81b440cedcb15d45...,技术总监 技术研发总监
3,1,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_pk/708c5a3d5f60edec...,技术总监 储能技术总监（欧美）
4,1,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_pk/a443110213bbe28b...,硬件技术总监 研发总监
...,...,...,...,...,...
1507,162,5026,衡阳市王中王家居集团有限公司,https://www.zhipin.com/job_pk/7f784b15deceec85...,创始人 项目合伙人
1508,162,5026,衡阳市王中王家居集团有限公司,https://www.zhipin.com/job_pk/f5c1f9c51c4dad0e...,展会公司总经理 联合创始人
1509,162,5026,衡阳市王中王家居集团有限公司,https://www.zhipin.com/job_pk/5b7534b3156cdd29...,联合创始人 股东合伙人
1510,162,5026,衡阳市王中王家居集团有限公司,https://www.zhipin.com/job_pk/51bfee7147a818df...,创始合伙人 高级合伙人


In [2]:
from MyUtil import Selenium_Edge
from MyUtil import OpenCv_Util
from selenium.webdriver.common.by import By
from pprint import pprint
from time import sleep
edge_driver = Selenium_Edge(False)
# 职位信息
job_original = []
# 对比职位的职位信息
compare_job_data = []
# 对比职位的公司信息
compare_company_data = []
# 加载失败链接
compare_fail_url = []
# 失效链接
compare_error_url = []
# 推荐对比
command_compare_data = []

start_compare = 819
end_compare = 1511
segment_name_compare = f"data_compare_{start_compare}_{end_compare}"  # 分段命名
segment_data_compare = data_compare[start_compare:end_compare]  # 获取当前分段的数据
segment_data_compare1 = pd.DataFrame(segment_data_compare)

下载目录:
D:\Path\study\Jupyter\crwal


In [3]:
if 1:
    previous_id = None
    number = start_compare
    for index_compare, data_compare in segment_data_compare1.iterrows():
        url = data_compare['command_compare_url']
        id = data_compare['id']
        company_name = data_compare['original_company']
        info_id ={
            "number" :number,
            "id":id,
            "company_name_original":company_name
        }
        if url == 'Null':
            compare_error_url.append(info_id)
            continue
        else:
            edge_driver.get_driver(url)
            info = {
                "number":number,
                "url":url,
                "id":id,
                "company_name_original":company_name,
                "compare_error_info":""
            }
            def collect_error():
                if edge_driver.get_element_text('//*[@id="main"]/div/div/div/a') == "首页":
                    info['compare_error_info'] = f"第{number}号对应的{id}的{company_name}对比链接失效"
                    print(info['compare_error_info'])
                    compare_error_url.append(info)
                    return True
                elif not edge_driver.wait_element(xpath='//*[@id="header"]/div[1]/div[2]/ul/li[1]/a',timeout=6,checktext="首页"):
                    info['compare_error_info'] = f"第{number}号对应的{id}的{company_name}相似链接加载失败"
                    print(info['compare_error_info'])
                    compare_fail_url.append(info)
                    return True
                elif edge_driver.get_element_attribute('//*[@id="wrap"]/div[3]/div/div[1]/div[1]/form/div[2]/p/input',attribute='placeholder') == "搜索职位、公司":
                    info['compare_error_info'] = f"第{number}号对应的{id}的{company_name}相似链接失效超时返回主页"
                    print(info['compare_error_info'])
                    compare_error_url.append(info)
                    return True
                else:
                    return False
            def get_company_compare():
                company_father = edge_driver.get_element(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[2]/div[2]')
                abbr_html_compare = edge_driver.get_element_attribute(father_element=company_father, xpath='./div/div/div/div[3]/div/span/a',attribute='href')
                company_name_compare = edge_driver.get_element_text(father_element=company_father, xpath='./div/div/div/div[3]/div/span/a')
                abbr_job_compare = edge_driver.get_element_attribute(father_element=company_father,xpath='./div/div/div/div[2]/h3/a',attribute='href')
                abbr_job_name_compare = edge_driver.get_element_text(father_element=company_father, xpath='./div/div/div/div[2]/h3/a')
                stage_compare = edge_driver.get_element_text(father_element=company_father, xpath="./div/div/div/div[3]/ul/li[2]")
                industry_compare = edge_driver.get_element_text(father_element=company_father, xpath="./div/div/div/div[3]/ul/li[1]")
                edge_driver.click_element('//*[@id="content"]/div/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[2]/h3/a',By.XPATH)
                salary_c = edge_driver.get_element_text(xpath='//*[@id="main"]/div[1]/div/div/div/div[2]/span')
                compare_job_company_compare = {
                        "number":number,
                        "id":id,
                        "compare_company_name":company_name_compare,
                        "compare_abbr_html":abbr_html_compare,
                        "compare_job_html":abbr_job_compare,
                        "compare_abbr_job_name":abbr_job_name_compare,
                        "compare_company_stage":stage_compare,
                        "compare_job_salary":salary_c,
                        "compare_company_industry":industry_compare
                    }
                pprint(compare_job_company_compare)
                if company_father == 'Null':
                    print(f"第{number}号对应的{id}公司简介为空！")
                else:
                    compare_company_data.append(compare_job_company_compare)
                edge_driver.get_driver(url)
                print(f"第{number}号对应的{id}的{company_name}职位信息爬取完毕")

            def get_job_introduce_compare():
                # 职位相关字段
                job_city = job_experience = job_degree = job_tags = job_keyword = job_requirements = job_benefits = company_name_compare = job_time_compare = ""
                company_name_compare = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[3]/div/span')
                job_father = edge_driver.get_element(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[1]/text()
                job_city = edge_driver.get_element_text(father_element=job_father, xpath='./ul[1]/li[1]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[2]/text()
                job_experience = edge_driver.get_element_text(father_element=job_father, xpath='./ul[1]/li[2]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[3]/text()
                job_degree = edge_driver.get_element_text(father_element=job_father, xpath='./ul[1]/li[3]')
                job_tags = edge_driver.get_element_text(father_element=job_father, xpath='./ul[2]/li')
                job_keyword = edge_driver.get_element_text(father_element=job_father, xpath='./p')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[4]/div[2]/div/ul[1]
                job_requirements = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[4]/div[2]/div/ul[1]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[7]/div[2]/div/ul/li
                job_benefits = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[7]/div[2]/div/ul/li')
                job_time_compare = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[6]/div[2]/div/div/div')
                job_data_compare = {
                    "number":number,
                    "id":id,
                    "compare_company_name":company_name_compare,
                    "compare_job_city":job_city,
                    "compare_job_experience":job_experience,
                    "command_job_degree":job_degree,
                    "command_job_tags":job_tags,
                    "command_job_keyword":job_keyword,
                    "command_job_requirements":job_requirements,
                    "command_job_time":job_time_compare,
                    "command_job_benefits":job_benefits
                }
                pprint(job_data_compare)
                if job_father == 'Null':
                    print(f"第{number}号对应的{id}的{company_name}简介为空！")
                else:
                    compare_job_data.append(job_data_compare)
                print(f"第{number}号对应的{id}的{company_name}职业信息已爬取完毕！")
            def get_recommand_compare():
                recommand_data = []
                url_father = edge_driver.get_elements("//div[@class='recommend']/div[2]")
                for i in url_father:
                    for j in range(1, 9):
                        recommand_name = edge_driver.get_element_attribute(xpath=f"./a[{j}]", father_element=i, attribute='href')
                        recommand_url = edge_driver.get_element_attribute(xpath=f"./a[{j}]", father_element=i)
                        recommand_data.append((recommand_name, recommand_url))
                for recommand in recommand_data:
                    command_compare = {
                        'number':number,
                        'id' : id,
                        'original_company' : company_name,
                        'command_compare_url' : recommand[0],
                        'command_compare_name' : recommand[1]
                    }
                    if recommand != 'Null':
                        command_compare_data.append(command_compare)
                print(f"第{number}号对应的{id}的{company_name}推荐数据爬取成功！")
            def get_original_job():
                # 职位相关字段
                job_city = job_experience = job_degree = job_tags = job_keyword = job_requirements = job_benefits = job_time = ""
                job_father = edge_driver.get_element(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[1]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[1]/div/ul[1]/li[1]
                job_city = edge_driver.get_element_text(father_element=job_father, xpath='./div/ul[1]/li[1]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[2]/text()
                job_experience = edge_driver.get_element_text(father_element=job_father, xpath='./div/ul[1]/li[2]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[3]/text()
                job_degree = edge_driver.get_element_text(father_element=job_father, xpath='./div/ul[1]/li[3]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[2]/li
                job_tags = edge_driver.get_element_text(father_element=job_father, xpath='./div/ul[2]/li')
                job_keyword = edge_driver.get_element_text(father_element=job_father, xpath='./div/p')
                job_requirements = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[4]/div[1]/div/ul[1]')
                job_time = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[6]/div[1]/div/div/div')
                job_benefits = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[7]/div[1]/div/ul')
                original_job_data = {
                    "number":number,
                    "id":id,
                    "original_company_name_c":company_name,
                    "original_job_city":job_city,
                    "original_job_experience":job_experience,
                    "original_job_degree":job_degree,
                    "original_job_tags":job_tags,
                    "original_job_keyword":job_keyword,
                    "original_job_requirements":job_requirements,
                    "original_job_time":job_time,
                    "original_job_benefits":job_benefits
                }
                pprint(original_job_data)
                if job_father != 'Null':
                    job_original.append(original_job_data)
                else:
                    compare_fail_url.append(info)
                    print(f"第{id}号的{company_name}职业信息加载失败！")
            if collect_error():
                continue
            elif id != previous_id:
                get_original_job()
            get_company_compare()
            get_job_introduce_compare()
            get_recommand_compare()
            previous_id = id
            number+=1

{'id': 5007,
 'number': 819,
 'original_company_name_c': '趣品（北京）电子商务股份有限公司',
 'original_job_benefits': 'Null',
 'original_job_city': '太原',
 'original_job_degree': '不限',
 'original_job_experience': '5-10年',
 'original_job_keyword': '有生产管理经验；对管理有独特见解。',
 'original_job_requirements': '管理经验\n生产管理',
 'original_job_tags': '人力资源管理',
 'original_job_time': 'Null'}
{'compare_abbr_html': 'https://www.zhipin.com/gongsi/14a0ab49f18bd0af1XV-2ti7FlA~.html',
 'compare_abbr_job_name': '坚果零食品牌总经理',
 'compare_company_industry': '餐饮',
 'compare_company_name': '串串叔叔美食有限公司',
 'compare_company_stage': 'Null',
 'compare_job_html': 'https://www.zhipin.com/job_detail/bd23cd9c5e287d981HZy0966E1pY.html',
 'compare_job_salary': '15-20K',
 'id': 5007,
 'number': 819}
第819号对应的5007的趣品（北京）电子商务股份有限公司职位信息爬取完毕
{'command_job_benefits': 'Null',
 'command_job_degree': '本科',
 'command_job_keyword': '1、负责本企业新品牌的全面负责\n2、需要做过炒货、坚果连锁品牌的，入股不是勿扰，谢谢',
 'command_job_requirements': 'None',
 'command_job_tags': '消费品',
 'command_job_ti

KeyboardInterrupt: 

In [4]:
import os
from MyUtil import OpenCv_Util
true_end = 1165
segment_name = f"{start_compare}_{true_end}"  # 分段命名
df_job_original_dir = f"./command_compare_output/job_original"
df_fail_url_dir =  f"./command_compare_output/compare_fail_url"
df_error_url_dir = f"./command_compare_output/compare_error_url"
df_job_data_compare_dir =  f"./command_compare_output/compare_job_data"
df_company_data_compare_dir =  f"./command_compare_output/compare_company_data"
df_job_list_data_compare_dir =  f"./command_compare_output/command_compare_data"
OpenCv_Util.mkdir(df_job_original_dir)
OpenCv_Util.mkdir(df_fail_url_dir)
OpenCv_Util.mkdir(df_error_url_dir)
OpenCv_Util.mkdir(df_job_data_compare_dir)
OpenCv_Util.mkdir(df_company_data_compare_dir)
OpenCv_Util.mkdir(df_job_list_data_compare_dir)

./command_compare_output/job_original文件夹已存在
./command_compare_output/compare_fail_url文件夹已存在
./command_compare_output/compare_error_url文件夹已存在
./command_compare_output/compare_job_data文件夹已存在
./command_compare_output/compare_company_data文件夹已存在
./command_compare_output/command_compare_data文件夹已存在


In [5]:
# 职位信息
df_job_original_data = pd.DataFrame(job_original)
print(df_job_original_data)
df_job_original_data.to_excel(f'{df_job_original_dir}/{segment_name}.xlsx', index=False)

   number    id original_company_name_c original_job_city  \
0     819  5007        趣品（北京）电子商务股份有限公司                太原   
1     824  5009             深圳市图派科技有限公司                天津   
2     864  5010            武汉云创未来科技有限公司                郴州   
3     904  5011            江苏四维智能科技有限公司                郴州   
4     944  5012            北京鑫盈信息咨询有限公司                绵阳   
5     984  5013           湖南京临轩餐饮管理有限公司                天津   
6    1024  5016            上海哈啰普惠科技有限公司                南京   
7    1063  5017            武汉中云康崇科技有限公司                合肥   
8    1103  5018           易虎网科技（集团）有限公司                合肥   
9    1143  5019            北京企课信息技术有限公司                合肥   

  original_job_experience original_job_degree original_job_tags  \
0                   5-10年                  不限            人力资源管理   
1                   5-10年                  本科              高级管理   
2                   5-10年                  大专             事业部管理   
3                   5-10年                  大专             事业

In [6]:
# 对比职位的职位信息
df_look_job_list_data = pd.DataFrame(compare_job_data)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_job_data_compare_dir}/{segment_name}.xlsx', index=False)

     number    id compare_company_name compare_job_city  \
0       819  5007           串串叔叔美食有限公司               太原   
1       820  5007           串串叔叔美食有限公司               太原   
2       821  5007            某大型交通上市公司               长沙   
3       822  5007                红海互联网              马鞍山   
4       823  5007               陕西恒丰集团               西安   
..      ...   ...                  ...              ...   
323    1161  5019              某医疗器械公司               天津   
324    1162  5019           串串叔叔美食有限公司               太原   
325    1163  5019           串串叔叔美食有限公司               太原   
326    1164  5019            某大型交通上市公司               长沙   
327    1165  5019                红海互联网              马鞍山   

    compare_job_experience command_job_degree command_job_tags  \
0                    5-10年                 本科              消费品   
1                    5-10年                 本科              消费品   
2                    10年以上                 本科              已上市   
3                    5-10年 

In [7]:
# 对比职位的公司信息
df_look_job_list_data = pd.DataFrame(compare_company_data)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_company_data_compare_dir}/{segment_name}.xlsx', index=False)

     number    id compare_company_name  \
0       819  5007           串串叔叔美食有限公司   
1       820  5007           串串叔叔美食有限公司   
2       821  5007                 Null   
3       822  5007                红海互联网   
4       823  5007               陕西恒丰集团   
..      ...   ...                  ...   
327    1161  5019                 Null   
328    1162  5019           串串叔叔美食有限公司   
329    1163  5019           串串叔叔美食有限公司   
330    1164  5019                 Null   
331    1165  5019                红海互联网   

                                     compare_abbr_html  \
0    https://www.zhipin.com/gongsi/14a0ab49f18bd0af...   
1    https://www.zhipin.com/gongsi/14a0ab49f18bd0af...   
2                                                 Null   
3    https://www.zhipin.com/gongsi/a67dc135f32f5910...   
4    https://www.zhipin.com/gongsi/9c874da0952ca126...   
..                                                 ...   
327                                               Null   
328  https://www.zhipin.com/gon

In [8]:
# 加载失败链接
df_look_job_list_data = pd.DataFrame(compare_fail_url)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_fail_url_dir}/{segment_name}.xlsx', index=False)

Empty DataFrame
Columns: []
Index: []


In [9]:
# 失效链接
df_look_job_list_data = pd.DataFrame(compare_error_url)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_error_url_dir}/{segment_name}.xlsx', index=False)

   number                                                url    id  \
0    1043  https://www.zhipin.com/job_pk/c25da5f771d678cc...  5016   

  company_name_original                      compare_error_info  
0          上海哈啰普惠科技有限公司  第1043号对应的5016的上海哈啰普惠科技有限公司相似链接失效超时返回主页  


In [10]:
# 推荐对比
df_look_job_list_data = pd.DataFrame(command_compare_data)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_job_list_data_compare_dir}/{segment_name}.xlsx', index=False)

      number    id  original_company  \
0        819  5007  趣品（北京）电子商务股份有限公司   
1        819  5007  趣品（北京）电子商务股份有限公司   
2        819  5007  趣品（北京）电子商务股份有限公司   
3        819  5007  趣品（北京）电子商务股份有限公司   
4        819  5007  趣品（北京）电子商务股份有限公司   
...      ...   ...               ...   
3227    1165  5019      北京企课信息技术有限公司   
3228    1165  5019      北京企课信息技术有限公司   
3229    1165  5019      北京企课信息技术有限公司   
3230    1165  5019      北京企课信息技术有限公司   
3231    1165  5019      北京企课信息技术有限公司   

                                    command_compare_url  \
0     https://www.zhipin.com/job_pk/4ce2f7468fca19ce...   
1     https://www.zhipin.com/job_pk/146b278aca7832d4...   
2     https://www.zhipin.com/job_pk/0803bdabd8bc323c...   
3     https://www.zhipin.com/job_pk/0db88beb3b036082...   
4     https://www.zhipin.com/job_pk/604609d321ef3f2a...   
...                                                 ...   
3227  https://www.zhipin.com/job_pk/0db88beb3b036082...   
3228  https://www.zhipin.com/job_pk/604609d321e

In [11]:
import pandas as pd
from MyUtil import OpenCv_Util
import os

# 定义一个函数来合并文件夹下的所有Excel文件
def merge_excel_files(folder_path):
    # 初始化一个空的DataFrame用于存储合并后的数据
    merged_df = pd.DataFrame()
    # 遍历文件夹中的所有文件
    for file in os.listdir(folder_path):
        # 检查文件扩展名是否为.xlsx（Excel文件）
        if file.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file)  # 获取文件完整路径
            df = pd.read_excel(file_path)  # 读取Excel文件
            merged_df = pd.concat([merged_df, df], ignore_index=True)  # 合并数据
    # 按照序号排序
    # merged_df.sort_values(by='number',inplace=True)
    return merged_df
df_job_original_dir = f"./command_compare_output/job_original"
df_fail_url_dir =  f"./command_compare_output/compare_fail_url"
df_error_url_dir = f"./command_compare_output/compare_error_url"
df_job_data_compare_dir =  f"./command_compare_output/compare_job_data"
df_company_data_compare_dir =  f"./command_compare_output/compare_company_data"
df_job_list_data_compare_dir =  f"./command_compare_output/command_compare_data"
df_dirs = []
df_dirs.append(df_job_original_dir)
df_dirs.append(df_fail_url_dir)
df_dirs.append(df_error_url_dir)
df_dirs.append(df_job_data_compare_dir)
df_dirs.append(df_company_data_compare_dir)
df_dirs.append(df_job_list_data_compare_dir)
# 创建一个新文件夹用于存储合并后的Excel文件
OpenCv_Util.mkdir('./command_compare_output/merge')
for df_dir in df_dirs:
    data = merge_excel_files(df_dir)
    df_dir = df_dir.replace('./command_compare_output/','')
    data.to_excel(f'./command_compare_output/merge/{df_dir}_merged.xlsx', index=False)
    print(df_dir,'处理完毕')

./command_compare_output/merge文件夹已存在
job_original 处理完毕
compare_fail_url 处理完毕
compare_error_url 处理完毕
compare_job_data 处理完毕
compare_company_data 处理完毕
command_compare_data 处理完毕


In [12]:
import pandas as pd
import os

def delete_rows_with_empty_third_column(file_path):
    df = pd.read_excel(file_path)
    df.dropna(subset=[df.columns[2]], inplace=True)
    df.to_excel(file_path, index=False)

def delete_empty_rows_in_xlsx_files(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file)
            delete_rows_with_empty_third_column(file_path)
folder_path = "./command_compare_output/merge/"
delete_empty_rows_in_xlsx_files(folder_path)
print("删除完毕！")

IndexError: index 2 is out of bounds for axis 0 with size 0